# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 21096922


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/32 [00:00<?, ?it/s]

Rendering models:   6%|▋         | 2/32 [00:02<00:42,  1.41s/it]

Rendering models:   9%|▉         | 3/32 [00:05<00:51,  1.77s/it]

Rendering models:  12%|█▎        | 4/32 [00:05<00:36,  1.30s/it]

Rendering models:  16%|█▌        | 5/32 [00:07<00:37,  1.38s/it]

...

Rendering models:  91%|█████████ | 29/32 [00:10<00:00,  7.15it/s]

Rendering models:  94%|█████████▍| 30/32 [00:10<00:00,  7.46it/s]

Rendering models:  97%|█████████▋| 31/32 [00:10<00:00,  6.26it/s]

Rendering models: 100%|██████████| 32/32 [00:10<00:00,  6.56it/s]

not-logged-in-3b0b929f4475fd276561    0.001806
Linda_J._Berkel                       0.000135
DrJackie                              0.000278
Jnursssmith5263                       0.000672
not-logged-in-7d01fb53d3c3d3999bca    0.000102
Jonas_Cross                           0.000139
awright5                              0.002178
not-logged-in-42aeec5653762437f6b7    0.000863
thiagoxxxx                            0.000112
TSM_Fatboy                            0.030440
MartyMcflurry                         0.004848
not-logged-in-2908c5b97527b9f7279b    0.000753
Galaxy-one                            0.000120
MakenzieHoward                        0.032796
sn346808                              0.036453
sn346808                              0.031098
sn346808                              0.008754
sn345012                              0.002665
ToriPizza                             0.000379
LeeJoseph                             0.009562
koshmarus                             0.000099
puic         

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())